<a href="https://colab.research.google.com/github/mayurs619/CS-572---Natural-Language-Processing/blob/main/proj_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 1: Language Modeling

*This assignment is adapted from one created by David Gaddy, Daniel Fried, Nikita Kitaev, Mitchell Stern, Rodolfo Corona, John DeNero, and Dan Klein.*

TA contact for this assignment: Saahas Parise (saahas.parise@duke.edu),
Xinchang Xiong (xinchang.xiong@duke.edu)


---



In this assignment, you will implement several different types of language models for text.  We'll start with n-gram models, then move on to neural n-gram and GRU language models.

**Warning**: Do not start this project the day before it is due!  Some parts require 20 minutes or more to run, so debugging and tuning can take a significant amount of time.

Our dataset for this project will be the WikiText2 language modeling dataset.  This dataset comes with some of the basic preprocessing done for us, such as tokenization and rare word filtering (using the `<unk>` token).
Therefore, we can assume that all word types in the test set also appear at least once in the training set.
We'll also use the Huggingface `datasets` and `tokenizers` libraries to help with some of the data preprocessing, such as converting tokens into id numbers.

**Note on GPU usage**: You will need to use a GPU for the neural n-gram and GRU models but **not** for the n-gram models. Colab places some restrictions on GPU usage due to which you might get locked out after continuously using one (~8 hours). To avoid this, you should only use the GPU when needed, i.e., on training and inference for the last two parts of this assignment. You can enable / disable GPU usage by changing the Runtime type under the Runtime menu.
If you do get locked out of using a GPU, a potential workaround is to sign in using a different account.

When training a model on the GPU it is also a good idea to save your model periodically in case you get locked out. You can use `torch.save(network.state_dict(), path)` and `network.load_state_dict()` for this; see [here](https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_models_for_inference.html). You can also save your `*.npy` files to google drive to avoid lossing them after Colab session cut-off (see the sample script below).

```
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/CS572-S24-A1
!cp *.npy /content/drive/MyDrive/CS572-S24-A1
!ls /content/drive/MyDrive/CS572-S24-A1
```

**Grading rubric**
- 70% results
 - 15% bigram_predictions.npy (correctness)
 - 15% trigram_kn_predictions.npy (correctness)
 - 15% neural_trigram_predictions.npy (meets target)
 - 15% gru_predictions.npy (meets target)
 - 10% gru_predictions.npy (improvement over target)
  
- 30% writeup
 - 12.5% clarity
 - 12.5% correctness
 - 5% interestingness of ideas

In [ ]:
# # Install some required packages.
!pip install transformers
!pip install datasets

In [ ]:
# This block handles some basic setup and data loading.
# You shouldn't need to edit this, but if you want to
# import other standard python packages, that is fine.

from collections import defaultdict, Counter
import numpy as np
import math
import tqdm
import random
import pdb

import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

# We'll use HuggingFace's datasets and tokenizers libraries, which are a bit
# heavy-duty for what we're doing, but it's worth getting to know them.

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import WhitespaceSplit

dataset = load_dataset("wikitext", "wikitext-2-v1")
tokenizer = Tokenizer(WordLevel(unk_token='<unk>'))
tokenizer.pre_tokenizer = WhitespaceSplit() # should be equivalent to split()

# "Training" a tokenizer below just feeds it all the tokens so it can map from
# word type to id.

# should only be 33,278 distinct types in Wikitext-2
trainer = WordLevelTrainer(vocab_size=33300, special_tokens=["<unk>", "<eos>"])
generator_bsz = 512
all_splits_generator = (dataset[split][i:i+generator_bsz]["text"]
                        for split in ["train", "validation", "test"]
                          for i in range (0, len(dataset[split]), generator_bsz))
tokenizer.train_from_iterator(all_splits_generator, trainer)

# If desired, we could make a transformers tokenizer object now with:
# fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)

orig_vocab = tokenizer.get_vocab() # The tokenizer reserves a <pad> id, which we'll ignore.
word_types = sorted(list(orig_vocab.keys()), key=lambda w: orig_vocab[w]) # no <pad>
vocab = {w: i for i, w in enumerate(word_types)} # no <pad>
vocab_size = len(vocab)

# Make a single stream of tokens, with an <eos> after each newline.

train_text = []
for example in dataset["train"]["text"]:
  train_text.extend(tokenizer.encode(example).tokens + ["<eos>"])

validation_text = []
for example in dataset["validation"]["text"]:
  validation_text.extend(tokenizer.encode(example).tokens + ["<eos>"])

print(validation_text[:30])

['<eos>', '=', 'Homarus', 'gammarus', '=', '<eos>', '<eos>', 'Homarus', 'gammarus', ',', 'known', 'as', 'the', 'European', 'lobster', 'or', 'common', 'lobster', ',', 'is', 'a', 'species', 'of', '<unk>', 'lobster', 'from', 'the', 'eastern', 'Atlantic', 'Ocean']


We've implemented a unigram model here as a demonstration.

In [ ]:
class UnigramModel:
    def __init__(self, train_text):
        self.counts = Counter(train_text)
        self.total_count = len(train_text)

    def probability(self, word):
        return self.counts[word] / self.total_count

    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""
        return [self.probability(word) for word in word_types]

    def perplexity(self, full_text):
        """Return the perplexity of the model on a text as a float.
        full_text -- a list of string tokens
        """
        log_probabilities = []
        for word in full_text:
            # Note that the base of the log doesn't matter as long as the log and exp use the same base.
            log_probabilities.append(math.log(self.probability(word), 2))
        return 2 ** -np.mean(log_probabilities)

unigram_demonstration_model = UnigramModel(train_text)
print('unigram validation perplexity:', unigram_demonstration_model.perplexity(validation_text))

def check_validity(model):
    """Performs several sanity checks on your model:
    1) That next_word_probabilities returns a valid distribution
    2) That perplexity matches a perplexity calculated from next_word_probabilities

    Although it is possible to calculate perplexity from next_word_probabilities,
    it is still good to have a separate more efficient method that only computes
    the probabilities of observed words.
    """

    log_probabilities = []
    for i in range(10):
        prefix = validation_text[:i]
        probs = model.next_word_probabilities(prefix)
        assert min(probs) >= 0, "Negative value in next_word_probabilities"
        assert max(probs) <= 1 + 1e-8, "Value larger than 1 in next_word_probabilities"
        assert abs(sum(probs)-1) < 1e-4, "next_word_probabilities do not sum to 1"

        word_id = vocab[validation_text[i]]
        selected_prob = probs[word_id]
        log_probabilities.append(math.log(selected_prob))

    perplexity = math.exp(-np.mean(log_probabilities))
    your_perplexity = model.perplexity(validation_text[:10])
    assert abs(perplexity-your_perplexity) < 0.1, "your perplexity does not " + \
    "match the one we calculated from `next_word_probabilities`,\n" + \
    "at least one of `perplexity` or `next_word_probabilities` is incorrect.\n" + \
    f"we calcuated {perplexity} from `next_word_probabilities`,\n" + \
    f"but your perplexity function returned {your_perplexity} (on a small sample)."


check_validity(unigram_demonstration_model)

unigram validation perplexity: 965.0860734119312


To generate from a language model, we can sample one word at a time conditioning on the words we have generated so far.

In [ ]:
def generate_text(model, n=20, prefix=('<eos>', '<eos>')):
    prefix = list(prefix)
    for _ in range(n):
        probs = model.next_word_probabilities(prefix)
        word = random.choices(word_types, probs)[0]
        prefix.append(word)
    return ' '.join(prefix)

print(generate_text(unigram_demonstration_model))

<eos> <eos> and spoke <unk> , <eos> win equals produced <unk> and Cabral flew but things placed either Reception she only to


In fact there are many strategies to get better-sounding samples, such as only sampling from the top-k words or sharpening the distribution with a temperature.  You can read more about sampling from a language model in this paper: https://arxiv.org/pdf/1904.09751.pdf.

You will need to submit some outputs from the models you implement for us to grade.  The following will download the required output files.

In [ ]:
!gdown --id 1aHC9RfmeSa8dDwC9XGQN8uRmtTwJaIRw
!gdown --id 1oedI437UeS9AhUGwsC-PgBdD1sHOjhaY

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1aHC9RfmeSa8dDwC9XGQN8uRmtTwJaIRw
To: /content/nu_eval_output_vocab_short.txt
100% 3.67k/3.67k [00:00<00:00, 12.5MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1oedI437UeS9AhUGwsC-PgBdD1sHOjhaY
To: /content/nu_eval_prefixes_short.txt
100% 104k/104k [00:00<00:00, 4.03MB/s]


In [ ]:
def save_truncated_distribution(model, filename, short=True):
    """Generate a file of truncated distributions.

    Probability distributions over the full vocabulary are large,
    so we will truncate the distribution to a smaller vocabulary.

    Please do not edit this function
    """
    vocab_name = 'nu_eval_output_vocab'
    prefixes_name = 'nu_eval_prefixes'

    if short:
      vocab_name += '_short'
      prefixes_name += '_short'

    with open('{}.txt'.format(vocab_name), 'r') as eval_vocab_file:
        eval_vocab = [w.strip() for w in eval_vocab_file]
    eval_vocab_ids = [vocab[s] for s in eval_vocab]

    all_selected_probabilities = []
    with open('{}.txt'.format(prefixes_name), 'r') as eval_prefixes_file:
        lines = eval_prefixes_file.readlines()
        #for line in tqdm.notebook.tqdm(lines, leave=False):
        for line in lines:
            prefix = line.strip().split(' ')
            probs = model.next_word_probabilities(prefix)
            selected_probs = np.array([probs[i] for i in eval_vocab_ids], dtype=np.float32)
            all_selected_probabilities.append(selected_probs)

    all_selected_probabilities = np.stack(all_selected_probabilities)
    np.save(filename, all_selected_probabilities)
    print('saved', filename)

In [ ]:
save_truncated_distribution(unigram_demonstration_model,
                            'unigram_demonstration_predictions.npy')

saved unigram_demonstration_predictions.npy


**Before you proceed**: At this point you should check whether you are able to upload the submission files to Gradescope. For this we will generate *dummy* prediction files by copying the unigram predictions above. Download the `unigrm_demonstration_predictions.npy` (you can do this by clicking the folder icon on left menu) and then copy this file and rename it to generate the required submision files:
* bigram_predictions.npy
* trigram_kn_predictions.npy
* neural_trigram_predictions.npy
* gru_predictions.npy

Also save a copy of this notebook as `proj_1.ipynb` and create a `report.pdf` (this can be empty for now). Upload these files to Gradescope and confirm that the autograder runs and produces an output score of 0.

### N-gram Model

Now it's time to implement an n-gram language model.

Because not every n-gram will have been observed in training, use add-alpha smoothing to make sure no output word has probability 0.

$$P(w_2|w_1)=\frac{\#(w_1,w_2)+\alpha}{\#(w_1)+V\alpha}$$

where $V$ is the vocab size and $\#()$ is the count for the given bigram.  An alpha value around `3e-3`  should work.  Later, we'll replace this smoothing with model backoff.

One edge case you will need to handle is at the beginning of the text where you don't have `n-1` prior words.  You can handle this however you like as long as you produce a valid probability distribution, but just using a uniform distribution over the vocabulary is reasonable for the purposes of this project.

A properly implemented bi-gram model should get a perplexity **below 505** on the validation set.

**Note**: Do not change the signature of the `next_word_probabilities` and `perplexity` functions.  We will use these as a common interface for all of the different model types.  Make sure these two functions call `n_gram_probability`, because later we are going to override `n_gram_probability` in a subclass.
Also, we suggest pre-computing and caching the counts $\#()$ when you initialize `NGramModel` for efficiency.

In [ ]:
class NGramModel:
    def __init__(self, train_text, n=2, alpha=3e-3):
        self.n = n
        self.smoothing = alpha

        self.ngram_counts = {}
        self.history_counts = {}

        for i in range(len(train_text) - n + 1):
            ngram = tuple(train_text[i : i+n])
            history = tuple(train_text[i : i+n-1])

            self.ngram_counts[ngram] = self.ngram_counts.get(ngram, 0) + 1
            self.history_counts[history] = self.history_counts.get(history, 0) + 1

    def _to_string(self, tokens):
        return "_".join(tokens)

    def n_gram_probability(self, n_gram):
        assert len(n_gram) == self.n

        numerator = self.ngram_counts.get(n_gram, 0) + self.smoothing

        if self.n == 1:
            denominator = len(train_text) + (self.smoothing * vocab_size)
        else:
            history = n_gram[:-1]
            denominator = self.history_counts.get(history, 0) + (self.smoothing * vocab_size)

        probability = numerator / denominator
        return probability

    def next_word_probabilities(self, text_prefix):
        if self.n == 1:
            return [self.n_gram_probability((word,)) for word in word_types]

        if len(text_prefix) > self.n-1:
            text_prefix = text_prefix[-(self.n-1):]

        if len(text_prefix) < self.n-1:
            return [1 / vocab_size] * vocab_size

        return [
            self.n_gram_probability(tuple(text_prefix) + (word,))
            for word in word_types
        ]

    def perplexity(self, full_text):
        log_probs = []

        for i in range(self.n-1):
            ngram = full_text[0 : i]
            prob = 1 / vocab_size
            log_probs.append(math.log(prob, 2))

        for i in range(self.n-1, len(full_text)):
            ngram = tuple(full_text[i-self.n+1 : i+1])
            prob = self.n_gram_probability(ngram)
            log_probs.append(math.log(prob, 2))

        perplexity = 2 ** -np.mean(log_probs)
        return perplexity

unigram_model = NGramModel(train_text, n=1)
check_validity(unigram_model)
print('unigram validation perplexity:', unigram_model.perplexity(validation_text))

bigram_model = NGramModel(train_text, n=2)
check_validity(bigram_model)
print('bigram validation perplexity:', bigram_model.perplexity(validation_text))

trigram_model = NGramModel(train_text, n=3)
check_validity(trigram_model)
print('trigram validation perplexity:', trigram_model.perplexity(validation_text))

save_truncated_distribution(bigram_model, 'bigram_predictions.npy')

unigram validation perplexity: 965.0913672756687
bigram validation perplexity: 504.4260671566396
trigram validation perplexity: 2965.602880102997
saved bigram_predictions.npy



Please download `bigram_predictions.npy` once you finish this section so that you can submit it.

In the block below, please report your bigram validation perplexity.  (We will use this to help us calibrate our scoring on the test set.)

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Bigram validation perplexity: ***504.4260671561098***

We can also generate samples from the model to get an idea of how it is doing.

In [ ]:
print(generate_text(bigram_model))

<eos> <eos> The Free Isesi outnumbered , although he had opposed to be a Bigger Protestants Vienna pitcher Copacabana accusing Taylor nevertheless


We now free up some RAM, **it is important to run the cell below, otherwise you will likely run out of RAM in the Colab runtime.**

In [ ]:
# Free up some RAM.
del bigram_model
del trigram_model

This basic model works okay for bigrams, but a better strategy (especially for higher-order models) is to use backoff.  Implement backoff with absolute discounting.
$$P\left(w_i|h_i\right)=\frac{max\left\{\#(h_i, w_i)-d,0\right\}}{\#(h_i)} + \lambda(h_i) P(w_i|w_{i-n+2},\ldots, w_{i-1})$$

$$\lambda\left(h_i\right)=\frac{d N_{1+}(h_i)}{{\#(h_i)}}$$
where $h_i=(w_{i-n+1}, \ldots, w_{i-1})$ is the prefix before token $i$, $N_{1+}$ is the number of words that appear after the previous $n-1$ words (the number of times the max will select something other than 0 in the first equation).  If $\#(h_i)=0$, use the lower order model probability directly (the above equations would have a division by 0).

We found a discount $d$ of 0.9 to work well based on validation performance.  A trigram model with this discount value should get a validation perplexity below 272.

In [ ]:
class DiscountBackoffModel(NGramModel):
    def __init__(self, train_text, lower_order_model, n=2, delta=0.9):
        super().__init__(train_text, n=n)
        self.lower_order_model = lower_order_model
        self.discount = delta

        self.ngram_counts = Counter(
            [tuple(train_text[i : i+self.n])
             for i in range(len(train_text) - self.n + 1)]
        )

        self.history_counts = Counter(
            [tuple(train_text[i:i+(self.n-1)])
             for i in range(len(train_text)-(self.n-1)+1)]
        )

        self.N1s = {}
        for i in range(len(train_text) - self.n + 1):
            ndx = i + self.n - 1
            prefix = tuple(train_text[i : ndx])
            next_word = train_text[ndx]
            if prefix in self.N1s:
                self.N1s[prefix].add(next_word)
            else:
                self.N1s[prefix] = {next_word}
        self.N1s = {k: len(v) for k, v in self.N1s.items()}

    def n_gram_probability(self, n_gram):
        assert len(n_gram) == self.n

        numerator = max(self.ngram_counts[n_gram] - self.discount, 0)
        denominator = self.history_counts[n_gram[:-1]]

        if denominator == 0:
            return self.lower_order_model.n_gram_probability(n_gram[1:])

        first_term = numerator / denominator

        prefix = n_gram[:-1]
        lambda_h1 = (self.discount * self.N1s[prefix]) / denominator
        second_term = lambda_h1 * self.lower_order_model.n_gram_probability(n_gram[1:])

        return first_term + second_term

bigram_backoff_model = DiscountBackoffModel(train_text, unigram_model, 2)
trigram_backoff_model = DiscountBackoffModel(train_text, bigram_backoff_model, 3)
check_validity(trigram_backoff_model)
print('trigram backoff validation perplexity:', trigram_backoff_model.perplexity(validation_text))

AttributeError: 'DiscountBackoffModel' object has no attribute 'context_counts'

Free up RAM.

In [ ]:
# Release models we don't need any more.
del unigram_model
del bigram_backoff_model
del trigram_backoff_model

Now, implement Kneser-Ney to replace the unigram base model.
$$P(w)\propto |\{w':\#(w',w) > 0\}|$$
A Kneser-Ney trigram model should get a validation perplexity below 257.

In [ ]:
class KneserNeyBaseModel(NGramModel):
    def __init__(self, train_text):
        super().__init__(train_text, n=1)

        self.ngram_counts = Counter(
            [tuple(train_text[i : i+self.n])
             for i in range(len(train_text) - self.n + 1)]
        )

        self.cache = {}
        for i in range(0, len(train_text)-2):
            prefix = tuple(train_text[i : i+1],)
            next_word = tuple(train_text[i+1 : i+2],)

            if i == 0:
                self.cache[prefix] = set()
            elif next_word in self.cache:
                self.cache[next_word].add(prefix)
            else:
                self.cache[next_word] = {prefix}

        self.cache = {k: len(v) for k, v in self.cache.items()}
        self.w_primes = sum(self.cache.values())

    def n_gram_probability(self, n_gram):
        assert len(n_gram) == 1
        return self.cache[n_gram] / self.w_primes

kn_base = KneserNeyBaseModel(train_text)
check_validity(kn_base)
bigram_kn_backoff_model = DiscountBackoffModel(train_text, kn_base, 2)
trigram_kn_backoff_model = DiscountBackoffModel(train_text, bigram_kn_backoff_model, 3)
print('trigram Kneser-Ney backoff validation perplexity:', trigram_kn_backoff_model.perplexity(validation_text))

save_truncated_distribution(trigram_kn_backoff_model, 'trigram_kn_predictions.npy') # this might take a few minutes

In [ ]:
print(generate_text(trigram_kn_backoff_model))
print(generate_text(trigram_kn_backoff_model, prefix=['What','about']))

Fill in your trigram backoff perplexities with and without Kneser Ney.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Trigram backoff validation perplexity: ***271.1216731108553***

Trigram backoff with Kneser Ney perplexity: ***256.61334330211224***

Free up RAM.

In [ ]:
# Delete models we don't need.
del kn_base
del bigram_kn_backoff_model
del trigram_kn_backoff_model

### Neural N-gram Model

In this section, you will implement a neural version of an n-gram model.  The model will use a simple feedforward neural network that takes the previous `n-1` words and outputs a distribution over the next word.

You will use PyTorch to implement the model.  We've provided a little bit of code to help with the data loading using PyTorch's data loaders (https://pytorch.org/docs/stable/data.html)

A model with the following architecture and hyperparameters should reach a validation perplexity **below or about 220**.
* embed the words with dimension 128, then flatten into a single embedding for $n-1$ words (with size $(n-1)*128$)
* run 2 hidden layers with 1024 hidden units, then project down to size 128 before the final layer (ie. 4 layers total).
* use weight tying for the embedding and final linear layer (this made a very large difference in our experiments); you can do this by creating the output layer with [`nn.Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html), then using [`F.embedding`](https://pytorch.org/docs/stable/generated/torch.nn.functional.embedding.html) with the linear layer's `.weight` to embed the input
* rectified linear activation (ReLU) and dropout 0.1 after first 2 hidden layers. **Note: You will likely find a performance drop if you add a nonlinear activation function after the dimension reduction layer.**
* train for 10 epochs with the Adam optimizer (should take around 15-20 minutes)
* do early stopping based on validation set perplexity (see Project 0)


We encourage you to try other architectures and hyperparameters, and you will likely find some that work better than the ones listed above.  A proper implementation with these should be enough to receive full credit on the assignment, although you might need to retrain the model for a few time.

In [ ]:
import torch.optim as optim
from tqdm import tqdm

def ids(tokens):
    return [vocab[t] for t in tokens]

assert torch.cuda.is_available(), "no GPU found, in Colab go to 'Edit->Notebook settings' and choose a GPU hardware accelerator"
device = torch.device("cuda")

class NeuralNgramDataset(torch.utils.data.Dataset):
    def __init__(self, text_token_ids, n):
        self.text_token_ids = text_token_ids
        self.n = n

    def __len__(self):
        return len(self.text_token_ids)

    def __getitem__(self, i):
        if i < self.n-1:
            prev_token_ids = [vocab['<eos>']] * (self.n-i-1) + self.text_token_ids[:i]
        else:
            prev_token_ids = self.text_token_ids[i-self.n+1:i]

        assert len(prev_token_ids) == self.n-1

        x = torch.tensor(prev_token_ids)
        y = torch.tensor(self.text_token_ids[i])
        return x, y

class NeuralNGramNetwork(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.n = n
        vocab_size = len(vocab)

        self.output_layer = nn.Linear(128, vocab_size)

        self.fc1 = nn.Linear((n-1)*128, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.projection = nn.Linear(1024, 128)

        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        embeds = F.embedding(x, self.output_layer.weight)
        embeds_flat = embeds.view(embeds.size(0), -1)

        out = self.fc1(embeds_flat)
        out = F.relu(out)
        out = self.dropout(out)

        out = self.fc2(out)
        out = F.relu(out)
        out = self.dropout(out)

        out = self.projection(out)

        logits = self.output_layer(out)
        log_probs = F.log_softmax(logits, dim=1)
        return log_probs

class NeuralNGramModel:
    def __init__(self, n):
        self.n = n
        self.network = NeuralNGramNetwork(n).to(device)

    def train(self, n_epochs=10):
        dataset = NeuralNgramDataset(ids(train_text), self.n)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)

        optimizer = optim.Adam(self.network.parameters(), lr=0.0005, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

        for epoch in range(n_epochs):
            self.network.train()
            total_loss = 0.0
            total_tokens = 0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
            for x, y in progress_bar:
                x, y = x.to(device), y.to(device)
                optimizer.zero_grad()
                log_probs = self.network(x)
                loss = F.nll_loss(log_probs, y, reduction='sum')
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                total_tokens += y.size(0)
                progress_bar.set_postfix({'loss': loss.item() / y.size(0)})

            avg_loss = total_loss / total_tokens
            train_ppl = math.exp(avg_loss)
            val_ppl = self.perplexity(validation_text)
            print(f"Epoch {epoch+1}: Train PPL: {train_ppl:.2f}, Val PPL: {val_ppl:.2f}")

            scheduler.step()

    def next_word_probabilities(self, text_prefix):
        self.network.eval()
        prefix_ids = ids(text_prefix)
        n = self.n
        if len(prefix_ids) < n - 1:
            padded = [vocab['<eos>']] * (n - 1 - len(prefix_ids)) + prefix_ids
        else:
            padded = prefix_ids[-(n-1):]
        x = torch.tensor(padded, device=device).unsqueeze(0)
        with torch.no_grad():
            log_probs = self.network(x)
        probs = torch.exp(log_probs).squeeze(0).cpu().numpy()
        return probs

    def perplexity(self, text):
        self.network.eval()
        dataset = NeuralNgramDataset(ids(text), self.n)
        loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=False)
        total_loss = 0.0
        total_tokens = 0

        with torch.no_grad():
            for x, y in loader:
                x, y = x.to(device), y.to(device)
                log_probs = self.network(x)
                loss = F.nll_loss(log_probs, y, reduction='sum')
                total_loss += loss.item()
                total_tokens += y.size(0)

        avg_loss = total_loss / total_tokens
        return math.exp(avg_loss)

neural_trigram_model = NeuralNGramModel(3)
check_validity(neural_trigram_model)
neural_trigram_model.train()
print('neural trigram validation perplexity:', neural_trigram_model.perplexity(validation_text))

save_truncated_distribution(neural_trigram_model, 'neural_trigram_predictions.npy')

Fill in your neural trigram perplexity.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Neural trigram validation perplexity: ***211.05881961163914***

Free up RAM.

In [ ]:
# Delete model we don't need.
del neural_trigram_model

### RNN (GRU) Model

For this stage of the project, you will implement an RNN language model using a popular variant -- [Gated recurrent unit (GRU)](https://en.wikipedia.org/wiki/Gated_recurrent_unit).

For recurrent language modeling, the data batching strategy is a bit different from what is used in some other tasks.  Sentences are concatenated together so that one sentence starts right after the other, and an unfinished sentence will be continued in the next batch.  We have provided a helper class `RecurrentLMDataset` to this for you.  To properly deal with this input format, you should save the last state of the GRU from a batch to feed in as the first state of the next batch.  When you save state across different batches, you should call `.detach()` on the state tensors before the next batch to tell PyTorch not to backpropagate gradients through the state into the batch you have already finished (which will cause a runtime error).

We expect your model to reach a validation perplexity **below 130**.  The following architecture and hyperparameters should be sufficient to get there.
* 2 GRU layers with 768 units
* dropout of 0.5 after each GRU layer and each linear layer (use different dropout modules)
* instead of projecting directly from the last GRU output to the vocabulary size for softmax, project down to a smaller size first (e.g. 768->128->vocab_size). **NOTE: You may find that adding nonlinearities between these layers can hurt performance, try without first.**
* use the same weights for the embedding layer and the pre-softmax layer; dimension 128
* train with Adam (using default learning rates) for at least 10 epochs
* clip gradient norms to be lower than 5 before taking an optimization step (code example below)


In [ ]:
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class RecurrentLMDataset:
    def __init__(self, text_token_ids, bsz, bptt_len=32):
        self.bsz = bsz
        self.bptt_len = bptt_len
        token_ids = torch.tensor(text_token_ids)
        ncontig = token_ids.size(0) // bsz
        token_ids = token_ids[:ncontig * bsz].view(bsz, -1)  # bsz x ncontig
        self.token_ids = token_ids.t().contiguous()  # ncontig x bsz

    def __len__(self):
        return int(math.ceil(self.token_ids.size(0) / self.bptt_len))

    def __iter__(self):
        for i in range(0, self.token_ids.size(0) - 1, self.bptt_len):
            seqlen = min(self.bptt_len, self.token_ids.size(0) - i - 1)
            x = self.token_ids[i:i + seqlen]      # seqlen x bsz
            y = self.token_ids[i + 1:i + seqlen + 1]  # seqlen x bsz
            yield x, y

# Hyperparameters
batch_size = 64
n_epochs = 10
bptt_len = 32
n_layers = 2
hidden_size = 768
embed_size = 128
clip = 5.0

class GRUNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.dropout_value = 0.5

        self.emb_dropout = nn.Dropout(self.dropout_value)

        self.gru1 = nn.GRU(input_size=self.embed_size,
                           hidden_size=self.hidden_size,
                           num_layers=1,
                           bias=True)
        self.gru1_dropout = nn.Dropout(self.dropout_value)

        self.gru2 = nn.GRU(input_size=self.hidden_size,
                           hidden_size=self.hidden_size,
                           num_layers=1,
                           bias=True)
        self.gru2_dropout = nn.Dropout(self.dropout_value)

        self.fc = nn.Linear(self.hidden_size, self.embed_size)
        self.fc_dropout = nn.Dropout(self.dropout_value)

        self.embedding_weight = nn.Parameter(torch.Tensor(self.vocab_size, self.embed_size))
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding_weight, -initrange, initrange)
        nn.init.uniform_(self.fc.weight, -initrange, initrange)
        nn.init.zeros_(self.fc.bias)

    def forward(self, x, state):
        embedded = F.embedding(x, self.embedding_weight)
        embedded = self.emb_dropout(embedded)

        out1, h1 = self.gru1(embedded, state[0:1])
        out1 = self.gru1_dropout(out1)

        out2, h2 = self.gru2(out1, state[1:2])
        out2 = self.gru2_dropout(out2)

        fc_out = self.fc(out2)
        fc_out = self.fc_dropout(fc_out)
        logits = F.linear(fc_out, self.embedding_weight)
        log_probs = F.log_softmax(logits, dim=-1)

        new_state = torch.cat([h1, h2], dim=0)
        new_state = new_state.detach()
        return log_probs, new_state

class GRUModel:
    def __init__(self):
        self.network = GRUNetwork().to(device)

    def train(self):
        optimizer = optim.Adam(self.network.parameters())
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                         factor=0.5, patience=1, verbose=True)
        train_dataset = RecurrentLMDataset(ids(train_text), batch_size, bptt_len)

        for epoch in range(n_epochs):
            self.network.train()
            total_loss = 0.0
            total_batches = 0
            state = torch.zeros(n_layers, batch_size, hidden_size, device=device)
            epoch_iterator = tqdm(train_dataset, total=len(train_dataset),
                                  desc=f"Epoch {epoch+1}/{n_epochs}")
            for x, y in epoch_iterator:
                x = x.to(device)
                y = y.to(device)
                optimizer.zero_grad()
                log_probs, state = self.network(x, state)
                loss = F.nll_loss(log_probs.view(-1, self.network.vocab_size), y.view(-1))
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.network.parameters(), clip)
                optimizer.step()

                total_loss += loss.item()
                total_batches += 1
                epoch_iterator.set_postfix(loss=loss.item())

            avg_train_loss = total_loss / total_batches
            val_ppl = self.perplexity(validation_text)
            print(f"Epoch {epoch+1}: Training Loss: {avg_train_loss:.4f} | Validation Perplexity: {val_ppl:.2f}")
            scheduler.step(val_ppl)
            current_lr = optimizer.param_groups[0]['lr']
            print(f"Current Learning Rate: {current_lr:.6f}")

    def next_word_probabilities(self, text_prefix):
        "Return a list of probabilities for each word in the vocabulary."
        self.network.eval()
        token_list = ids(text_prefix)
        if len(token_list) == 0:
            raise ValueError("Text prefix is empty. Please provide a non-empty prefix.")
        token_ids = torch.tensor(token_list, dtype=torch.long, device=device).unsqueeze(1)
        state = torch.zeros(n_layers, 1, hidden_size, device=device)
        with torch.no_grad():
            for i in range(token_ids.size(0)):
                out, state = self.network(token_ids[i:i+1], state)
            next_word_log_probs = out[-1, 0, :]
            next_word_probs = torch.exp(next_word_log_probs)
        return next_word_probs.cpu().numpy().tolist()

    def perplexity(self, text):
        "Return perplexity (a float) computed over the provided text."
        self.network.eval()
        val_dataset = RecurrentLMDataset(ids(text), batch_size, bptt_len)
        total_loss = 0.0
        total_tokens = 0
        state = torch.zeros(n_layers, batch_size, hidden_size, device=device)
        with torch.no_grad():
            for x, y in tqdm(val_dataset, total=len(val_dataset), desc="Evaluating"):
                x = x.to(device)
                y = y.to(device)
                out, state = self.network(x, state)
                loss = F.nll_loss(out.view(-1, self.network.vocab_size),
                                  y.view(-1),
                                  reduction='sum')
                total_loss += loss.item()
                total_tokens += y.numel()
        avg_loss = total_loss / total_tokens
        return math.exp(avg_loss)

gru_model = GRUModel()
gru_model.train()

print('gru validation perplexity:', gru_model.perplexity(validation_text))
save_truncated_distribution(gru_model, 'gru_predictions.npy')

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Fill in your GRU perplexity.

GRU validation perplexity: ***126.03939457275233***

# Experimentation: 1-Page Report

Now it's time for you to experiment.  Try to reach a validation perplexity **below 120**. You may either modify the GRU class above, or copy it down to the code cell below and modify it there. Just **be sure to run code cell below to generate results with your improved GRU**.

**NOTE:** We will award at least 7 of the 10 points if your improved model reaches a perplexity below 140 on the hidden test cases on Gradescope.

It is okay if the bulk of your improvements are due to hyperparameter tuning (such as changing the number or size of layers, adding dropout to more outputs), but implement at least one more substantial change to the model.  Here are some ideas (several of which come from https://arxiv.org/pdf/1708.02182.pdf), they do not always work, so might need to experiment with multiple ideas before seeing an improvement:
* weight-drop regularization - apply dropout to the weight matrices instead of activations
* learning rate scheduling - decrease the learning rate during training
* weight regularization - add a l2 regularization penalty on the weights of the networks
* ensembling - average the predictions of several models trained with different initialization random seeds
* embedding dropout - zero out the entire embedding for a random set of words in the embedding matrix
* activation regularization - add a l2 regularization penalty on the activation of the GRU output
* temporal activation regularization - add l2 regularization on the difference between the GRU output activations at adjacent timesteps

You may notice that most of these suggestions are regularization techniques. This dataset is considered fairly small, so regularization is one of the best ways to improve performance.

For this section, you will submit a write-up describing the extensions and/or modifications that you tried.  Your write-up should be **1-page maximum** in length and should be submitted in PDF format.  You may use any editor you like, but we recommend using LaTeX and working in an environment like Overleaf.
For full credit, your write-up should include:
1.   A concise and precise description of the extension(s) that you tried.
2.   A motivation for why you believed this approach might improve your model.
3.   A discussion of whether the extension was effective and/or an analysis of the results.  This will generally involve some combination of tables, learning curves, etc.
4.   A bottom-line summary of your results comparing validation perplexities of your improvement to the original GRU.
The purpose of this exercise is to experiment, so feel free to try/ablate multiple of the suggestions above as well as any others you come up with!
When you submit the file, please name it `report.pdf`.



Run the cell below in order to train your improved GRU and evaluate it.  

In [ ]:
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class RecurrentLMDataset:
    def __init__(self, text_token_ids, bsz, bptt_len=32):
        self.bsz = bsz
        self.bptt_len = bptt_len
        token_ids = torch.tensor(text_token_ids)
        ncontig = token_ids.size(0) // bsz
        token_ids = token_ids[:ncontig * bsz].view(bsz, -1)  # bsz x ncontig
        self.token_ids = token_ids.t().contiguous()  # ncontig x bsz

    def __len__(self):
        return int(math.ceil(self.token_ids.size(0) / self.bptt_len))

    def __iter__(self):
        for i in range(0, self.token_ids.size(0) - 1, self.bptt_len):
            seqlen = min(self.bptt_len, self.token_ids.size(0) - i - 1)
            x = self.token_ids[i:i + seqlen]      # seqlen x bsz
            y = self.token_ids[i + 1:i + seqlen + 1]  # seqlen x bsz
            yield x, y

# Hyperparameters
batch_size = 64
n_epochs = 15
bptt_len = 32
n_layers = 2
hidden_size = 768
embed_size = 128
clip = 5.0

class GRUNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.dropout_value = 0.4

        self.emb_dropout = nn.Dropout(self.dropout_value)

        self.gru1 = nn.GRU(input_size=self.embed_size,
                           hidden_size=self.hidden_size,
                           num_layers=1,
                           bias=True)
        self.gru1_dropout = nn.Dropout(self.dropout_value)

        self.gru2 = nn.GRU(input_size=self.hidden_size,
                           hidden_size=self.hidden_size,
                           num_layers=1,
                           bias=True)
        self.gru2_dropout = nn.Dropout(self.dropout_value)

        self.fc = nn.Linear(self.hidden_size, self.embed_size)
        self.fc_dropout = nn.Dropout(self.dropout_value)

        self.embedding_weight = nn.Parameter(torch.Tensor(self.vocab_size, self.embed_size))
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding_weight, -initrange, initrange)
        nn.init.uniform_(self.fc.weight, -initrange, initrange)
        nn.init.zeros_(self.fc.bias)

    def forward(self, x, state):
        embedded = F.embedding(x, self.embedding_weight)
        embedded = self.emb_dropout(embedded)

        out1, h1 = self.gru1(embedded, state[0:1])
        out1 = self.gru1_dropout(out1)

        out2, h2 = self.gru2(out1, state[1:2])
        out2 = self.gru2_dropout(out2)

        fc_out = self.fc(out2)
        fc_out = self.fc_dropout(fc_out)
        logits = F.linear(fc_out, self.embedding_weight)
        log_probs = F.log_softmax(logits, dim=-1)

        new_state = torch.cat([h1, h2], dim=0)
        new_state = new_state.detach()
        return log_probs, new_state

class GRUModel:
    def __init__(self):
        self.network = GRUNetwork().to(device)

    def train(self):
        optimizer = optim.Adam(self.network.parameters())
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                         factor=0.5, patience=1, verbose=True)
        train_dataset = RecurrentLMDataset(ids(train_text), batch_size, bptt_len)

        for epoch in range(n_epochs):
            self.network.train()
            total_loss = 0.0
            total_batches = 0
            state = torch.zeros(n_layers, batch_size, hidden_size, device=device)
            epoch_iterator = tqdm(train_dataset, total=len(train_dataset),
                                  desc=f"Epoch {epoch+1}/{n_epochs}")
            for x, y in epoch_iterator:
                x = x.to(device)
                y = y.to(device)
                optimizer.zero_grad()
                log_probs, state = self.network(x, state)
                loss = F.nll_loss(log_probs.view(-1, self.network.vocab_size), y.view(-1))
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.network.parameters(), clip)
                optimizer.step()

                total_loss += loss.item()
                total_batches += 1
                epoch_iterator.set_postfix(loss=loss.item())

            avg_train_loss = total_loss / total_batches
            val_ppl = self.perplexity(validation_text)
            print(f"Epoch {epoch+1}: Training Loss: {avg_train_loss:.4f} | Validation Perplexity: {val_ppl:.2f}")
            scheduler.step(val_ppl)
            current_lr = optimizer.param_groups[0]['lr']
            print(f"Current Learning Rate: {current_lr:.6f}")

    def next_word_probabilities(self, text_prefix):
        "Return a list of probabilities for each word in the vocabulary."
        self.network.eval()
        token_list = ids(text_prefix)
        if len(token_list) == 0:
            raise ValueError("Text prefix is empty. Please provide a non-empty prefix.")
        token_ids = torch.tensor(token_list, dtype=torch.long, device=device).unsqueeze(1)
        state = torch.zeros(n_layers, 1, hidden_size, device=device)
        with torch.no_grad():
            for i in range(token_ids.size(0)):
                out, state = self.network(token_ids[i:i+1], state)
            next_word_log_probs = out[-1, 0, :]
            next_word_probs = torch.exp(next_word_log_probs)
        return next_word_probs.cpu().numpy().tolist()

    def perplexity(self, text):
        "Return perplexity (a float) computed over the provided text."
        self.network.eval()
        val_dataset = RecurrentLMDataset(ids(text), batch_size, bptt_len)
        total_loss = 0.0
        total_tokens = 0
        state = torch.zeros(n_layers, batch_size, hidden_size, device=device)
        with torch.no_grad():
            for x, y in tqdm(val_dataset, total=len(val_dataset), desc="Evaluating"):
                x = x.to(device)
                y = y.to(device)
                out, state = self.network(x, state)
                loss = F.nll_loss(out.view(-1, self.network.vocab_size),
                                  y.view(-1),
                                  reduction='sum')
                total_loss += loss.item()
                total_tokens += y.numel()
        avg_loss = total_loss / total_tokens
        return math.exp(avg_loss)

gru_model = GRUModel()
gru_model.train()

print('gru validation perplexity:', gru_model.perplexity(validation_text))
save_truncated_distribution(gru_model, 'gru_predictions.npy')

### Submission

Upload a submission with the following files to Gradescope:
* proj_1.ipynb (rename to match this exactly)
* gru_predictions.npy (this should also include all improvements from your exploration)
* neural_trigram_predictions.npy
* trigram_kn_predictions.npy
* bigram_predictions.npy
* report.pdf

You can upload files individually or as part of a zip file, but if using a zip file be sure you are zipping the files directly and not a folder that contains them.

Be sure to check the output of the autograder after it runs.  It should confirm that no files are missing and that the output files have the correct format.  Note that the test set perplexities shown by the autograder are on a different scale from your validation set perplexities due to selecting different text and truncating the distribution.  Don't worry if the values seem worse. We will compare your perplexity on the test set to our model's perplexity and assign a score based on that.